In [54]:
import sys
import pandas as pd
import numpy as np

data = pd.read_csv('/Users/alston/Documents/ML/train.csv', encoding = 'big5')
data = data.iloc[:, 3:]

data[data == 'NR'] = 0
raw_data = data.to_numpy() #將資料以row為根據放進array中

month_data = {}
sample = np.empty([18, 480])
for month in range(12):   
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample
    print(month_data[month])
    
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value

mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
            
            
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w


testdata = pd.read_csv('/Users/alston/Documents/ML/test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y


import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

[[14.   14.   14.   ... 14.   13.   13.  ]
 [ 1.8   1.8   1.8  ...  1.8   1.8   1.8 ]
 [ 0.51  0.41  0.39 ...  0.34  0.41  0.43]
 ...
 [35.   79.    2.4  ... 48.   63.   53.  ]
 [ 1.4   1.8   1.   ...  1.1   1.9   1.9 ]
 [ 0.5   0.9   0.6  ...  1.2   1.2   1.3 ]]
[[ 15.    14.    14.   ...   8.4    8.     7.6 ]
 [  1.8    1.8    1.7  ...   1.7    1.7    1.7 ]
 [  0.27   0.26   0.25 ...   0.36   0.35   0.32]
 ...
 [113.   109.   104.   ...  72.    65.    69.  ]
 [  2.3    2.2    2.6  ...   1.9    2.9    1.5 ]
 [  2.5    2.2    2.2  ...   0.9    1.6    1.1 ]]
[[ 18.    18.    18.   ...  14.    13.    13.  ]
 [  1.8    1.8    1.8  ...   1.8    1.8    1.8 ]
 [  0.39   0.36   0.4  ...   0.42   0.47   0.49]
 ...
 [103.   128.   115.   ...  60.    94.    53.  ]
 [  1.7    1.4    1.8  ...   4.2    3.5    4.3 ]
 [  1.9    0.8    1.5  ...   3.1    2.4    2.4 ]]
[[ 19.    18.    17.   ...  24.    24.    23.  ]
 [  1.7    1.7    1.7  ...   1.8    1.8    1.9 ]
 [  0.42   0.42   0.42 ...   0.41   0.

['id_196', 33.2569641504973]
['id_197', 9.274804963637084]
['id_198', 23.07980707686087]
['id_199', 51.746694617979124]
['id_200', 20.160120634921274]
['id_201', -32.51048447374452]
['id_202', 64.1062591590245]
['id_203', 10.824126683242515]
['id_204', 2.0730111224714776]
['id_205', 8.613957516794155]
['id_206', 15.622314173038319]
['id_207', -1.9060287283284296]
['id_208', 125.83039290324942]
['id_209', 12.789873729899652]
['id_210', 12.527469419233224]
['id_211', 11.02385588248502]
['id_212', 32.58004546705762]
['id_213', 38.33322902589592]
['id_214', 0.5671586839433527]
['id_215', 34.494405506168796]
['id_216', 80.2723817430571]
['id_217', -2.9282997662477115]
['id_218', 10.078351824425908]
['id_219', 39.50748704508321]
['id_220', 10.138131252687469]
['id_221', 14.29296071943808]
['id_222', 104.59014204809613]
['id_223', 19.834101322187692]
['id_224', 16.800270882415326]
['id_225', 53.14642130871346]
['id_226', 13.044965884370587]
['id_227', 53.30773353317467]
['id_228', 2.067350691

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


array([[ 1.        ,  1.58612872,  1.59670557, ..., -0.6100614 ,
        -1.10245951,  0.3796299 ],
       [ 1.        , -0.52772648, -0.83033327, ...,  0.49935528,
         0.13093343, -0.36127762],
       [ 1.        ,  4.90790118,  4.63050411, ..., -1.47294105,
        -1.10245951, -0.48476221],
       ...,
       [ 1.        ,  2.79404598,  3.11360484, ...,  0.6226238 ,
         0.00759414,  1.86144494],
       [ 1.        , -1.43366443, -1.43709298, ..., -1.10313549,
        -1.22579881, -1.22566973],
       [ 1.        , -0.52772648, -0.52695342, ...,  4.1974109 ,
         4.57114804,  2.60235246]])